# Yolo8 Training Scripts

conda info --envs

conda create --name env1 python==3.12.7

conda activate env1

In [2]:
# !pip install pydantic==1.10.7
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
# !pip install ultralytics
# !pip install opencv-python-headless
# !pip install matplotlib
# !pip install tensorboard torch-tb-profiler

# !pip show torch torchvision
# !nvcc --version

### 1. Dataset folder structure.

dataset/
<br>│
<br>├── test/
<br>│   ├── images/
<br>│   ├── labels/
<br>│
<br>├── train/
<br>│   ├── images/
<br>│   ├── labels/
<br>│
<br>├── valid/
<br>│   ├── images/
<br>│   ├── labels/

<br>

### 2. Config file

data.yaml

<br>train: ../Training Day 6/train/images
<br>val: ../Training Day 6/valid/images
<br>test: ../Training Day 6/test/images
<br>
<br>nc: 13
<br>names: ['Cycling Helmet', 'half face', 'hard hat', 'helmet', 'modular helmet', 'motorbike', 'motorcyclist', 'no helmet', 'nutshell', 'person', 'plate', 'quarter face helmet', 'sports helmet']


In [4]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # Choose a YOLOv8 model variant (e.g., 'yolov8n.pt', 'yolov8s.pt')

# Train the model with TensorBoard logging enabled
model.train(
    data='C:/Users/winai/OneDrive - Advanced Information Technology Public Company Limited/All Data/My AI/AI Python/Training Day 6/data.yaml',
    epochs=100,
    batch=16,
    imgsz=640,
    # device=0,
    project='runs',  # Directory where TensorBoard logs will be saved
    name='yolov8_experiment',  # Sub-directory for this experiment
    exist_ok=True  # Overwrites the logs if already exist
)

New https://pypi.org/project/ultralytics/8.3.28 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.27  Python-3.12.7 torch-2.5.1+cu124 CPU (12th Gen Intel Core(TM) i7-1255U)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:/Users/winai/OneDrive - Advanced Information Technology Public Company Limited/All Data/My AI/AI Python/Training Day 6/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs, name=yolov8_experiment, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False

train: Scanning C:\Users\winai\OneDrive - Advanced Information Technology Public Company Limited\All Data\My AI\AI Python\Training Day 6\train\labels... 210 images, 0 backgrounds, 0 corrupt: 100%|██████████| 210/210 [00:00<00:00, 1380.16it/s]


train: New cache created: C:\Users\winai\OneDrive - Advanced Information Technology Public Company Limited\All Data\My AI\AI Python\Training Day 6\train\labels.cache


val: Scanning C:\Users\winai\OneDrive - Advanced Information Technology Public Company Limited\All Data\My AI\AI Python\Training Day 6\valid\labels... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<00:00, 1642.41it/s]

val: New cache created: C:\Users\winai\OneDrive - Advanced Information Technology Public Company Limited\All Data\My AI\AI Python\Training Day 6\valid\labels.cache


...

                   all         20         38      0.995          1      0.995      0.881



100 epochs completed in 2.581 hours.
Optimizer stripped from runs\yolov8_experiment\weights\last.pt, 6.3MB
Optimizer stripped from runs\yolov8_experiment\weights\best.pt, 6.3MB

Validating runs\yolov8_experiment\weights\best.pt...
Ultralytics 8.3.27  Python-3.12.7 torch-2.5.1+cu124 CPU (12th Gen Intel Core(TM) i7-1255U)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<00:00,  2.74s/it]


                   all         20         38      0.996          1      0.995      0.894
                   mom         20         20      0.995          1      0.995      0.916
               moodeng         18         18      0.997          1      0.995      0.873
Speed: 4.2ms preprocess, 122.9ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\yolov8_experiment


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001E37C63AF60>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

# Inference

In [21]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random

classes_map = {}
classes_map[0] = "mom"
classes_map[1] = "moodeng"

# Load a model
model = YOLO('../Training Day 6/runs/yolov8_experiment/weights/best.pt')

obj_classes = list(model.names.values())
classes_ids = [obj_classes.index(clas) for clas in obj_classes]
_colors = [random.choices(range(256), k=3) for _ in classes_ids]

imgs = [
    '../Training Day 6/Upload/frame_32198.jpg',
    '../Training Day 6/Upload/frame_37446.jpg',
    #'../Training Day 6/Upload/frame_50279.jpg',
    #'../Training Day 6/Upload/frame_56716.jpg',
    #'../Training Day 6/Upload/frame_63891.jpg',
    #'C:/Users/winai/OneDrive - Advanced Information Technology Public Company Limited/All Data/My AI/AI Python/Training Day 6/Upload/frame_28016.jpg',
    #'C:/Users/winai/OneDrive - Advanced Information Technology Public Company Limited/All Data/My AI/AI Python/Training Day 6/Upload/frame_24039.jpg',
    #'C:/Users/winai/OneDrive - Advanced Information Technology Public Company Limited/All Data/My AI/AI Python/Training Day 6/Upload/frame_25146.jpg',
    #'C:/Users/winai/OneDrive - Advanced Information Technology Public Company Limited/All Data/My AI/AI Python/Training Day 6/Upload/frame_29123.jpg',
    #'C:/Users/winai/OneDrive - Advanced Information Technology Public Company Limited/All Data/My AI/AI Python/Training Day 6/Upload/frame_39414.jpg',
    #'C:/Users/winai/OneDrive - Advanced Information Technology Public Company Limited/All Data/My AI/AI Python/Training Day 6/Upload/frame_54092.jpg',
    #'C:/Users/winai/OneDrive - Advanced Information Technology Public Company Limited/All Data/My AI/AI Python/Training Day 6/Upload/frame_54994.jpg',
    #'C:/Users/winai/OneDrive - Advanced Information Technology Public Company Limited/All Data/My AI/AI Python/Training Day 6/Upload/frame_63850.jpg',
    #'C:/Users/winai/OneDrive - Advanced Information Technology Public Company Limited/All Data/My AI/AI Python/Training Day 6/Upload/frame_64014.jpg',
    #'C:/Users/winai/OneDrive - Advanced Information Technology Public Company Limited/All Data/My AI/AI Python/Training Day 6/Upload/frame_83038.jpg',
]

for img in imgs:
    frame = cv2.imread(img)

    results = model.predict(frame, conf=0.2, show_labels=True, vid_stride=3, verbose=False)
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy.squeeze().tolist()
            confidence = box.conf.squeeze().tolist()
            color_number = classes_ids.index(int(box.cls[0]))
            label = classes_map[color_number]
            color = _colors[color_number]
            cv2.rectangle(frame, (int(x1),int(y1)), (int(x2),int(y2)),color,2)
            cv2.putText(frame, f"{round(confidence,2)} {label}", (int(x1), int(y1)-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)

    plt.figure(figsize=(100, 100))
    plt.imshow(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))
    #plt.show()  # แสดงผลลัพธ์
    #plt.close()  # ปิด figure หลังแสดงผลเสร็จ